In [21]:
import csv
import os
import json
import time
import random


In [22]:
llm_plan="""1. Navigate to the profile settings to update your bio with information that reflects your passion for plant research.
2. Explore the forums to identify and bookmark those relevant to botanical discussions to easily access them later.
3. Adjust notification preferences to ensure you receive alerts for posts related to botany advancements and plant science breakthroughs."""

llm_plan = "Adjust notification preferences to ensure you receive alerts for posts related to botany advancements and plant science breakthroughs."

now_page="""Tab 0 (current): Postmill

[1] RootWebArea 'Postmill' focused: True
	[32] HeaderAsNonLandmark ''
		[33] link 'Home'
	[65] link 'Forums'
	[66] link 'Wiki'
	[74] searchbox 'Search query'
	[75] link 'Notifications (0)'
	[76] link 'Submit'
	[12] button 'MarvelsGrantMan136' hasPopup: menu expanded: False
	[14] main ''
		[181] link 'Submissions'
		[182] link 'Comments'
		[18] button 'Filter on: Featured' hasPopup: menu expanded: False
		[20] button 'Sort by: Hot' hasPopup: menu expanded: False
		[55] StaticText 'You are not subscribed to any forum. Showing featured forums instead.'
	[26] complementary ''
		[56] heading 'Featured forums'
		[82] StaticText 'There are no featured forums to display.'
	[23] contentinfo ''
		[59] StaticText 'Running '
		[60] link 'Postmill'"""

now_url="http://reddit.com/"

In [23]:
with open("/home/zjusst/qms/webarena/result_stage_1_explore_v2/flitered_triplets.csv", "r") as f:
    reader = csv.reader(f)
    # 跳过第一行
    next(reader)
    triplets = [row for row in reader]
print(len(triplets))
print(triplets[0])

1028
["('812_1.json', 0, 0, 'http://reddit.com/')", "click [65] where [65] is link 'Forums'", 'The user intends to navigate to the list of available forums to explore different communities or topics.', "('812_1.json', 0, 1, 'http://reddit.com/forums')"]


In [24]:
from rank_bm25 import BM25Okapi
def bm25_retrieval(query, corpus, top_n=3):
    tokenized_corpus = [doc.split(" ") for doc in corpus]
    bm25 = BM25Okapi(tokenized_corpus)
    tokenized_query = query.split(" ")
    top_n_docs = bm25.get_top_n(tokenized_query, corpus, n=top_n)
    top_n_docs_index = [corpus.index(doc) for doc in top_n_docs]
    return top_n_docs_index

In [25]:
intention_corpus = [t[2] for t in triplets]
indexs = bm25_retrieval(llm_plan, intention_corpus, top_n=3)
for i in indexs:
    print(triplets[i])

["('812_2.json', 0, 1, 'http://reddit.com/notifications')", "type [6915] [explore notification settings ] where [6915] is searchbox 'Search query'", 'The user wants to find information or discussions related to notification settings, possibly seeking advice or solutions about how to manage notification settings on the platform.', "('812_44.json', 14, 1, 'http://reddit.com/search?q=explore+notification+settings')"]
["('812_42.json', 15, 1, 'http://reddit.com/f/boston/103483/i-ve-made-the-list-of-nearly-everything-going-on-in-boston')", "click [9924] where [9924] is button 'Subscribe No subscribers'", 'The user wants to subscribe to the forum to receive updates or follow discussions related to Boston events.', "('812_42.json', 16, 1, 'http://reddit.com/f/boston/103483/i-ve-made-the-list-of-nearly-everything-going-on-in-boston')"]
["('812_11.json', 3, 1, 'http://reddit.com/search?q=AI+advancements')", "click [5562] where [5562] is link 'If you were performing a Turing test to a super adva

In [31]:
page_positions[0]

"('812_1.json', 0, 0, 'http://reddit.com/')"

In [37]:
page_positions = set()
for triplet in triplets:
    page_positions.add(triplet[0])
    page_positions.add(triplet[3])

# 转为 list
page_positions = list(page_positions)
page_corpus = []

for page_position in page_positions:
    file_name, traj_index, pos, real_url = eval(page_position)
    with open(f"/home/zjusst/qms/webarena/result_stage_1_explore_v2/add_local_intention_trajs/{file_name}", "r") as f:
        trajs = json.load(f)
    traj = trajs[traj_index]
    if pos == 0:
        a11y = traj["a11y_before"]
    else:
        a11y = traj["a11y_after"]
    page_corpus.append(a11y)


In [39]:
indexs = bm25_retrieval(now_page, page_corpus, top_n=3)
for i in indexs:
    print(page_positions[i])
    print(page_corpus[i])
    


('812_1.json', 0, 0, 'http://reddit.com/')
Tab 0 (current): Postmill

[1] RootWebArea 'Postmill' focused: True
	[32] HeaderAsNonLandmark ''
		[33] link 'Home'
	[65] link 'Forums'
	[66] link 'Wiki'
	[74] searchbox 'Search query'
	[75] link 'Notifications (0)'
	[76] link 'Submit'
	[12] button 'MarvelsGrantMan136' hasPopup: menu expanded: False
	[14] main ''
		[181] link 'Submissions'
		[182] link 'Comments'
		[18] button 'Filter on: Featured' hasPopup: menu expanded: False
		[20] button 'Sort by: Hot' hasPopup: menu expanded: False
		[55] StaticText 'You are not subscribed to any forum. Showing featured forums instead.'
	[26] complementary ''
		[56] heading 'Featured forums'
		[82] StaticText 'There are no featured forums to display.'
	[23] contentinfo ''
		[59] StaticText 'Running '
		[60] link 'Postmill'
('812_5.json', 0, 1, 'http://reddit.com/')
Tab 0 (current): Postmill

[1] RootWebArea 'Postmill' focused: True
	[32] HeaderAsNonLandmark ''
		[33] link 'Home'
	[65] link 'Forums'
	[66]

In [ ]:
import networkx as nx

def find_trajectories_to_intent(graph_data, start_page_id, target_intent):
    """
    在图中查找从起始页面到包含特定意图的边的所有最短轨迹。

    Args:
        graph_data (list of tuples): 你的图数据，格式为 (source_id, action, intent, target_id)。
        start_page_id (any): 你当前所在的页面ID。
        target_intent (str): 你希望达到的边的意图。

    Returns:
        list of lists: 一个包含所有可能轨迹的列表。
                       每个轨迹本身是一个步骤列表，格式为 (page_id, action, intent)。
    """
    # 1. 构建一个有向图。我们使用 MultiDiGraph，因为它允许两个页面间存在多条不同的边。
    G = nx.MultiDiGraph()
    for source, action, intent, target in graph_data:
        # 将动作和意图作为边的属性存储
        G.add_edge(source, target, action=action, intent=intent)

    # 2. 找到所有匹配目标意图的“候选边”
    candidate_edges = []
    for u, v, data in G.edges(data=True):
        if data.get('intent') == target_intent:
            candidate_edges.append({'source': u, 'target': v, 'data': data})
    
    if not candidate_edges:
        print(f"图中未找到意图为 '{target_intent}' 的边。")
        return []

    # 3. 对于每个候选边，查找从起点到该边起点的最短路径
    all_found_trajectories = []
    for edge in candidate_edges:
        path_destination_node = edge['source']
        
        try:
            # 检查是否存在路径，并计算最短路径（节点列表）
            node_path = nx.shortest_path(G, source=start_page_id, target=path_destination_node)
            
            # 4. 如果找到路径，构建完整的轨迹
            trajectory = []
            # 遍历路径中的每一步（除最后一步外）
            for i in range(len(node_path) - 1):
                u, v = node_path[i], node_path[i+1]
                # 获取这两个节点间边的信息（这里我们取第一条，因为是最短路径）
                edge_data = G.get_edge_data(u, v)[0] 
                trajectory.append((u, edge_data['action'], edge_data['intent']))
            
            # 添加最后的目标边，完成轨迹
            final_step = (edge['source'], edge['data']['action'], edge['data']['intent'])
            trajectory.append(final_step)
            
            # 添加最终到达的页面
            trajectory.append((edge['target'], None, None)) 

            all_found_trajectories.append(trajectory)

        except nx.NetworkXNoPath:
            # 如果从起点到该边的起点没有路径，则忽略
            continue
        except nx.NodeNotFound:
            print(f"错误：起始节点 '{start_page_id}' 不在图中。")
            return []


    return all_found_trajectories

# ==================== 示例用法 ====================
if __name__ == "__main__":
    # 你的图数据
    graph_tuples = [
        ('PageA', 'click_products', 'navigate_to_products', 'PageB'),
        ('PageB', 'click_item1', 'view_item_details', 'PageC'),
        ('PageB', 'click_item2', 'view_item_details', 'PageD'),
        ('PageC', 'add_to_cart', 'add_item_to_cart', 'PageE'),
        ('PageD', 'add_to_cart', 'add_item_to_cart', 'PageE'),
        ('PageF', 'click_login', 'navigate_to_login', 'PageG'), # 孤立的图部分
    ]
    
    current_page = 'PageA'
    desired_intent = 'add_item_to_cart'
    
    print(f"从 {current_page} 开始，寻找意图为 '{desired_intent}' 的路径...\n")
    
    trajectories = find_trajectories_to_intent(graph_tuples, current_page, desired_intent)
    
    if trajectories:
        print(f"找到了 {len(trajectories)} 条可能的轨迹：")
        for i, traj in enumerate(trajectories):
            print(f"\n--- 轨迹 {i+1} ---")
            for step in traj:
                page, action, intent = step
                if action:
                    print(f"  在页面 '{page}' -> 执行动作 '{action}' (意图: {intent})")
                else:
                    print(f"  最终到达页面 '{page}'")
    else:
        print("未找到可行的轨迹。")